# Road Following - Live demo

In this notebook, we will use model we trained to move jetBot smoothly on track. 

### Load Trained Model

We will assume that you have already downloaded ``best_steering_model_xy.pth`` to work station as instructed in "train_model.ipynb" notebook. Now, you should upload model file to JetBot in to this notebooks's directory. Once that's finished there should be a file named ``best_steering_model_xy.pth`` in this notebook's directory.

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model. This should look very familiar from the training notebook.

In [ ]:
import torchvision
import torch
from yolov5.object_detect import run as object_detect_inference
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.torch_utils import select_device, smart_inference_mode
from yolov5.utils.general import scale_coords, non_max_suppression

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

labels_to_names = {0:'Stop_Sign'}
ob_device = select_device('cuda:0')
ob_data = '/home/woonho/python/autonomous_project/dataset/data.yaml'
ob_weights = 'best.pt'
ob_model = DetectMultiBackend(weights, device=ob_device, data=ob_data)

Next, load the trained weights from the ``best_steering_model_xy.pth`` file that you uploaded.

In [ ]:
model.load_state_dict(torch.load('best_steering_model_xy_0920.pth'))

<All keys matched successfully>

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [ ]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

### Creating the Pre-Processing Function

We have now loaded our model, but there's a slight issue. The format that we trained our model doesnt exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps:

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

def ob_preprocess(image):
    image = np.array(image)
    image = torch.from_numpy(image).to(device)
    image = image.half() if model.fp16 else image.float()  # uint8 to fp16/32
    image /= 255
    image = image.permute(2, 0, 1)
    transform = transforms.Resize(imgsz)
    image = transform(image)
    return image

Awesome! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera. You should be pretty familiar with this by now. 

In [ ]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

We'll also create our robot instance which we'll need to drive the motors.

In [ ]:
from jetbot import Robot

robot = Robot()

Now, we will define sliders to control JetBot
> Note: We have initialize the slider values for best known configurations, however these might not work for your dataset, therefore please increase or decrease the sliders according to your setup and environment

1. Speed Control (speed_gain_slider): To start your JetBot increase ``speed_gain_slider`` 
2. Steering Gain Control (steering_gain_sloder): If you see JetBot is woblling, you need to reduce ``steering_gain_slider`` till it is smooth
3. Steering Bias control (steering_bias_slider): If you see JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets

> Note: You should play around above mentioned sliders with lower speed to get smooth JetBot road following behavior.

Next, let's display some sliders that will let us see what JetBot is thinking.  The x and y sliders will display the predicted x, y values.

The steering slider will display our estimated steering value.  Please remember, this value isn't the actual angle of the target, but simply a value that is
nearly proportional.  When the actual angle is ``0``, this will be zero, and it will increase / decrease with the actual angle.  

In [ ]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

Next, we'll create a function that will get called whenever the camera's value changes. This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. Compute the approximate steering value
4. Control the motors using proportional / derivative control (PD)

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower should work, but the neural network is only as good as the data it's trained on!

In [ ]:
from time import time
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_offset = 0.34
    
    angle_diff_bias = 0.1
    angle = np.arctan2(x, y)
    if abs(angle) + angle_diff_bias < abs(angle_last):
        angle_temp = -1 * angle * 0.3
    else:
        angle_temp = angle
    speed_value = speed_offset - abs(angle_temp) * 0.05
    
    if abs(angle_temp) < 0.05:
        angle_temp = 0
    
    angle_last = angle
    
    angle_temp *= 1.4
    
    # Yolo v5
    pred = model(ob_preprocess(image), augment=False)
    pred = non_max_suppression(pred, conf_thres = 0.25, iou_thres = 0.45, classes = None, agnostic_nms = False, max_det=1000)
    if len(pred[0]) > 0:
      robot.left_motor.value = 0.0
      robot.right_motor.value = -angle_temp
    else
      robot.left_motor.value = speed_value
      robot.right_motor.value = -angle_temp
    
execute({'new': camera.value})
camera.observe(execute, names='value')

0.30452410051679396
0.29816518505523154
0.2950139992257813
0.3103262858182611
0.30773159011474915
0.3380801317418185
0.3505982708574632
0.36210069784882376
0.39091009439889496
0.4164764615754672
0.44432829962978737
0.47812779787688353
0.5027535455082108
0.5283659147017028
0.5667936629703463
0.5940727922876705
0.6170211630812265
0.6335803760291618
0.6931386431918116
0.7251669621493556
0.749815595829756
0.7684449447264271
0.7799998844110585
0.7681655314325452
0.7226622621955452
0.5906959546327162
0.5811757579887664
0.642954062895994
0.6248180829464401
0.5586617782561268
0.5253109001155505
0.43902137985722084
0.3413253402935839
0.31067589957271463
0.27770729214539763
0.2912560315815481
0.2575976625489311
0.1149611799313757
-0.10126211792866426
-0.31832681217789577
-0.30194854838722107
-0.28498991123687084
-0.3137406076612433
-0.3936168944945381
-0.41497348886573754
-0.41856954673850055
-0.41377313170995117
-0.508724519219478
-0.6150923080096651
-0.5933621649541168
-0.5588336249627055
-0.5

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame. 

You can now place JetBot on  Lego or Track you have collected data on and see whether it can follow track.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [ ]:

import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

-1.1183538129639705
-1.1270961963479982
-1.1537665358966425


### Conclusion
That's it for this live demo! Hopefully you had some fun seeing your JetBot moving smoothly on track follwing the road!!!

If your JetBot wasn't following road very well, try to spot where it fails. The beauty is that we can collect more data for these failure scenarios and the JetBot should get even better :)

###### 